In [ ]:
import pickle
from functools import partial
import numpy as np
from scipy.stats import pearsonr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import StratifiedShuffleSplit
from google.colab import files, drive

### 1 - Useful Functions

In [ ]:
# Useful functions for ploting figures of paper and implementation RMSE metric.

def history_plot(model1, model2, title1, title2):

    fig, (ax1, ax2) = plt.subplots(1, 2)

    fig.set_figheight(6)
    fig.set_figwidth(12)

    ax1.plot(model1.epoch, model1.history['loss'],
             color='forestgreen', label='Train')
    ax1.plot(model1.epoch, model1.history['val_loss'],
             color='lightcoral', label='Validation')
    ax1.set_xlabel('Epochs', fontsize=12)
    ax1.set_ylabel('Loss', fontsize=12)
    ax1.set_title(label=title1, size=14)
    ax1.grid(True, ls='--')
    ax1.legend()

    ax2.plot(model2.epoch, model2.history['loss'],
             color='forestgreen', label='Train')
    ax2.plot(model2.epoch, model2.history['val_loss'],
             color='lightcoral', label='Validation')
    ax2.set_xlabel('Epochs', fontsize=12)
    ax2.set_ylabel('Loss', fontsize=12)
    ax2.set_title(label=title2, size=14)
    ax2.grid(True, ls='--')
    ax2.legend()

    plt.savefig('history_curve.png', dpi=300)
    plt.show()


def correlation_plot(y_true, y_pred):

    ax = sns.jointplot(y_true, y_pred, facecolor='lightsteelblue',
                       edgecolor='c', marginal_kws={'color': 'darkcyan'})
    ax.set_axis_labels('Experimental', 'Predicted', fontsize=13)
    ax.fig.suptitle('Test set $R_{P}$ = 0.79', fontsize=14)
    ax.fig.tight_layout()
    ax.fig.subplots_adjust(top=0.9)

    plt.savefig('correlation_plot.png', dpi=300)
    plt.show()


def bar_chart(input, colormap, xlabel, filename):

    my_cmap = plt.cm.get_cmap(colormap)
    plt.figure(figsize=(6, 10))
    plt.barh(range(len(input) + 1, 1, -1), np.array(input)
             [:, 1], color=my_cmap(range(1, (len(input) + 1) * 4, 4)))
    plt.yticks(range(len(input) + 1, 1, -1),
               np.array(input, dtype=int)[:, 0], fontsize=10)
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel('Target Cluster ID', fontsize=12)
    plt.tight_layout()

    plt.savefig(filename, dpi=300)
    plt.show()


def rmse(y_true, y_pred):
    dev = np.square(y_true.ravel() - y_pred.ravel())
    return np.sqrt(np.sum(dev) / y_true.shape[0])

### 2 - Loading Data

#### PDBbind 2016v. dataset

In [ ]:
# opening refined set data.

with open('refined_set_data.pickle', 'rb') as handle:
    refined_set_data = pickle.load(handle)

# opening general minus refined set data.

with open('general_minus_refined_set_data.pickle', 'rb') as handle:
    general_minus_refined_set_data = pickle.load(handle)

In [ ]:
# loading pKd values for genral, refined and core set data.

general_set = pd.read_csv('general_set_binding_data.csv', index_col=0)
refined_set = pd.read_csv('refined_minus_core_set_binding_data.csv')
core_set = pd.read_csv('core_set_binding_data.csv', index_col=0)

In [ ]:
# some pdbid in general set don't have structures. Following codes find out these missing structures.

g_s_m = set(list(refined_set_data.keys()) +
            list(general_minus_refined_set_data.keys()))
g_s = set(general_set['pdbid'].to_list())
missing = list(g_s.difference(g_s_m))

In [ ]:
# droping missing data from genral set

general_set = general_set.set_index('pdbid')
general_set.drop(missing, axis=0, inplace=True)
general_set.reset_index(inplace=True)

In [ ]:
# making training and validation set

core_set_pdbid = core_set['pdbid'].to_list()

general_set = general_set.set_index('pdbid')

general_set.drop(core_set_pdbid, axis=0, inplace=True)

general_set.reset_index(inplace=True)

general_set['ba_cat'] = np.ceil(general_set['binding_affinity']/1.5)

general_set['ba_cat'].where(general_set['ba_cat'] < 8, 8, inplace=True)

split = StratifiedShuffleSplit(n_splits=1, test_size=0.077, random_state=42)

for train_index, val_index in split.split(general_set, general_set['ba_cat']):
    strat_train_set = general_set.loc[train_index]
    strat_val_set = general_set.loc[val_index]

In [ ]:
# pdbid of train, validation and test(core) set

strat_train_set_pdbid = strat_train_set['pdbid'].to_list()
strat_val_set_pdbid = strat_val_set['pdbid'].to_list()
core_set_pdbid = core_set['pdbid'].to_list()

In [ ]:
# making input and target for train, validation and test(core) set

strat_train_set_x = []
strat_val_set_x = []

for key in strat_train_set_pdbid:
    try:
        strat_train_set_x.append(general_minus_refined_set_data[key])
    except Exception:
        strat_train_set_x.append(refined_set_data[key])

for key in strat_val_set_pdbid:
    try:
        strat_val_set_x.append(general_minus_refined_set_data[key])
    except Exception:
        strat_val_set_x.append(refined_set_data[key])

strat_train_set_x = np.array(strat_train_set_x).reshape(
    len(strat_train_set_pdbid), 60, 100, 1)
strat_val_set_x = np.array(strat_val_set_x).reshape(
    len(strat_val_set_pdbid), 60, 100, 1)
test_set_x = np.array([refined_set_data[key]
                      for key in core_set_pdbid]).reshape(285, 60, 100, 1)

strat_train_set_y = strat_train_set['binding_affinity'].to_numpy()
strat_val_set_y = strat_val_set['binding_affinity'].to_numpy()
test_set_y = core_set['binding_affinity'].to_numpy()

In [ ]:
# shape of train, validation and test(core) set

print('X Train: ', strat_train_set_x.shape)
print('X Val: ', strat_val_set_x.shape)
print('X Test: ', test_set_x.shape)
print('Y Train: ', strat_train_set_y.shape)
print('Y Val: ', strat_val_set_y.shape)
print('Y Test: ', test_set_y.shape)

### 3 - Model

#### 3-1 Residual Model

In [ ]:
# Implemention Residual model using Tensorflow functional API.

def resiual_module(x, filters):

    conv1 = tf.keras.layers.Conv2D(
        filters, 3, 2, padding='SAME', use_bias=False)(x)
    batch1 = tf.keras.layers.BatchNormalization()(conv1)
    conv2 = tf.keras.layers.Conv2D(
        filters, 3, 1, padding='SAME', use_bias=False)(batch1)
    conv3 = tf.keras.layers.Conv2D(filters, kernel_size=1, strides=2)(x)
    batch2 = tf.keras.layers.BatchNormalization()(conv3)
    activation = tf.keras.activations.get('relu')

    return activation(tf.add(conv2, batch2))


def create_residual_functional_model(input_size, dropout=0.1):

    input = tf.keras.layers.Input(input_size)
    conv1 = tf.keras.layers.Conv2D(32, 3, 1, activation='relu')(input)
    conv1 = tf.keras.layers.BatchNormalization()(conv1)
    pool1 = tf.keras.layers.MaxPool2D(
        pool_size=2, strides=2, padding='SAME')(conv1)
    for filters in [64, 64, 128, 128, 256]:
        res = residual_module(pool1, filters)
        pool1 = res
    glob1 = tf.keras.layers.GlobalAvgPool2D()(res)
    dense1 = tf.keras.layers.Dense(100, activation='relu')(glob1)
    batch1 = tf.keras.layers.BatchNormalization()(dense1)
    drop1 = tf.keras.layers.Dropout(dropout)(batch1)
    dense2 = tf.keras.layers.Dense(50, activation='relu')(drop1)
    batch2 = tf.keras.layers.BatchNormalization()(dense2)
    drop2 = tf.keras.layers.Dropout(dropout)(batch2)
    output = tf.keras.layers.Dense(1)(drop2)

    model = tf.keras.models.Model(inputs=[input], outputs=[output])

    return model

In [ ]:
# Implemention Residual model using Tensorflow custom layer and sequential API.

DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding='SAME', use_bias=False)


class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.strides = strides
        self.activation = activation
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                keras.layers.BatchNormalization()]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'filters': self.filters, 'strides': self.strides,
                'activation': self.activation}


def create_residual_custom_model(input_size, dropout=0.1):

    model = keras.models.Sequential()
    # model.add(keras.layers.BatchNormalization())
    model.add(DefaultConv2D(32, input_shape=input_size))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.MaxPool2D(pool_size=2, strides=2, padding='SAME'))
    for filters in [64, 64, 128, 128, 256]:
        model.add(ResidualUnit(filters, strides=2))
    model.add(keras.layers.GlobalAvgPool2D())
    model.add(keras.layers.Dense(100, activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(50, activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(1))

    return model

#### 3-2 Sequential Model

In [ ]:
# Implemenation of Sequential model using Tensorflow sequential API.

def create_sequential_model(input_size, dropout=0.1):

    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, kernel_size=4, strides=1,
                               padding='valid', input_shape=input_size, activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='same'),

        tf.keras.layers.Conv2D(64, 4, 1, padding='valid', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='same'),

        tf.keras.layers.Conv2D(128, 4, 1, padding='valid', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='same'),

        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(
            400, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout),

        tf.keras.layers.Dense(
            200, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout),

        tf.keras.layers.Dense(
            100, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout),

        tf.keras.layers.Dense(
            20, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout),

        tf.keras.layers.Dense(1),
    ])

    return model

#### 3-3 Inception Model

In [ ]:
# Implemenation of Inception model using Tensorflow functional API.

def inception_module(x,
                     filters_1,
                     filters_3_reduce,
                     filters_3,
                     filters_5_reduce,
                     filters_5,
                     filters_pool):

    conv1 = tf.keras.layers.Conv2D(
        filters_1, 1, padding='same', activation='relu')(x)

    conv3 = tf.keras.layers.Conv2D(
        filters_3_reduce, 1, padding='same', activation='relu')(x)
    conv3 = tf.keras.layers.Conv2D(
        filters_3, 3, padding='same', activation='relu')(conv3)

    conv5 = tf.keras.layers.Conv2D(
        filters_5_reduce, 1, padding='same', activation='relu')(x)
    conv5 = tf.keras.layers.Conv2D(
        filters_5, 5, padding='same', activation='relu')(conv5)

    pool = tf.keras.layers.MaxPool2D(3, 1, padding='same')(x)
    conv6 = tf.keras.layers.Conv2D(
        filters_pool, 1, padding='same', activation='relu')(pool)

    output = tf.keras.layers.concatenate([conv1, conv3, conv5, conv6], axis=3)

    return output


def create_inception_model(input_size, dropout=0.1):

    input = tf.keras.layers.Input(shape=input_size)

    conv1 = tf.keras.layers.Conv2D(
        32, 4, 1, padding='same', activation='relu')(input)
    pool1 = tf.keras.layers.MaxPool2D(2, 2, padding='same')(conv1)

    inception1 = inception_module(pool1, 64, 96, 128, 16, 32, 32)
    inception2 = inception_module(inception1, 128, 128, 192, 32, 96, 64)

    glob1 = tf.keras.layers.GlobalAveragePooling2D()(inception2)

    dense1 = tf.keras.layers.Dense(
        400, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='relu')(glob1)
    batch1 = tf.keras.layers.BatchNormalization()(dense1)
    drop1 = tf.keras.layers.Dropout(dropout)(batch1)

    dense2 = tf.keras.layers.Dense(
        200, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='relu')(drop1)
    batch2 = tf.keras.layers.BatchNormalization()(dense2)
    drop2 = tf.keras.layers.Dropout(dropout)(batch2)

    dense3 = tf.keras.layers.Dense(
        100, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='relu')(drop2)
    batch3 = tf.keras.layers.BatchNormalization()(dense3)
    drop3 = tf.keras.layers.Dropout(dropout)(batch3)

    dense4 = tf.keras.layers.Dense(
        20, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='relu')(drop3)
    batch4 = tf.keras.layers.BatchNormalization()(dense4)
    drop4 = tf.keras.layers.Dropout(dropout)(batch4)

    output = tf.keras.layers.Dense(1)(drop4)

    model = tf.keras.models.Model(input, output)

    return model

### 4 - Training

#### 4-1 Residual Model

In [ ]:
# Compiling model

residual_model = create_residual_custom_model((60, 100, 1))

es = tf.keras.callbacks.EarlyStopping(
    patience=40, monitor='val_mse', restore_best_weights=True)
adam = tf.keras.optimizers.Adam(lr=0.0001)
residual_model.compile(optimizer=adam, loss='mse', metrics=['mse'])

In [ ]:
# Training Residual model

residual_history = residual_model.fit(strat_train_set_x, strat_train_set_y, batch_size=128, epochs=500,
                                      validation_data=(strat_val_set_x, strat_val_set_y), callbacks=[es])

In [ ]:
residual_model.save('residual_model.h5')

In [ ]:
y_pred_test = residual_model.predict(test_set_x)
y_pred_val = residual_model.predict(strat_val_set_x)

In [ ]:
# calculating metrics for residual model

loss_test = residual_model.evaluate(test_set_x, test_set_y, verbose=0)
loss_val = residual_model.evaluate(strat_val_set_x, strat_val_set_y, verbose=0)

pcc_test = pearsonr(test_set_y, y_pred_test.ravel())[0]
rmse_test = rmse(test_set_y, y_pred_test.ravel())

pcc_val = pearsonr(strat_val_set_y, y_pred_val.ravel())[0]
rmse_val = rmse(strat_val_set_y, y_pred_val.ravel())

print('-------------------')
print('Residual Model')
print(f'Test Loss: {loss_test[1]:.3f} Val Loss: {loss_val[1]:.3f}')
print(f'Test PCC: {pcc_test:.3f} Val PCC: {pcc_val:.3f}')
print(f'Test RMSE: {rmse_test:.3f} Val RMSE: {rmse_val:.3f}')

#### 4-2 Sequential Model

In [ ]:
# Compiling model

sequential_model = create_sequential_model((60, 100, 1))

es = tf.keras.callbacks.EarlyStopping(
    patience=40, monitor='val_mse', restore_best_weights=True)
adam = tf.keras.optimizers.Adam(lr=0.0001)
sequential_model.compile(optimizer=adam, loss='mse', metrics=['mse'])

In [ ]:
# Training Sequential model

sequential_history = sequential_model.fit(strat_train_set_x, strat_train_set_y, batch_size=128, epochs=500,
                                          validation_data=(strat_val_set_x, strat_val_set_y), callbacks=[es])

In [ ]:
sequential_model.save('sequential_model.h5')

In [ ]:
y_pred_test = sequential_model.predict(test_set_x)
y_pred_val = sequential_model.predict(strat_val_set_x)

In [ ]:
# Calculating metrics for Sequential model.

loss_test = sequential_model.evaluate(test_set_x, test_set_y, verbose=0)
loss_val = sequential_model.evaluate(
    strat_val_set_x, strat_val_set_y, verbose=0)

pcc_test = pearsonr(test_set_y, y_pred_test.ravel())[0]
rmse_test = rmse(test_set_y, y_pred_test.ravel())

pcc_val = pearsonr(strat_val_set_y, y_pred_val.ravel())[0]
rmse_val = rmse(strat_val_set_y, y_pred_val.ravel())

print('-------------------')
print('Sequential Model')
print(f'Test Loss: {loss_test[1]:.3f} Val Loss: {loss_val[1]:.3f}')
print(f'Test PCC: {pcc_test:.3f} Val PCC: {pcc_val:.3f}')
print(f'Test RMSE: {rmse_test:.3f} Val RMSE: {rmse_val:.3f}')

#### 4-3 Inception Model

In [ ]:
# Compiling model

inception_model = create_inception_model((60, 100, 1))

es = tf.keras.callbacks.EarlyStopping(
    patience=40, monitor='val_mse', restore_best_weights=True)
adam = tf.keras.optimizers.Adam(lr=0.0001)
inception_model.compile(optimizer=adam, loss='mse', metrics=['mse'])

In [ ]:
# Training Inception model.

inception_history = inception_model.fit(strat_train_set_x, strat_train_set_y, batch_size=128, epochs=500,
                                        validation_data=(strat_val_set_x, strat_val_set_y), callbacks=[es, history_cb])

In [ ]:
inception_model.save('inception_model.h5')

In [ ]:
y_pred_test = inception_model.predict(test_set_x)
y_pred_val = inception_model.predict(strat_val_set_x)

In [ ]:
# Calculating metrics for Inception model.

loss_test = inception_model.evaluate(test_set_x, test_set_y, verbose=0)
loss_val = inception_model.evaluate(
    strat_val_set_x, strat_val_set_y, verbose=0)

pcc_test = pearsonr(test_set_y, y_pred_test.ravel())[0]
rmse_test = rmse(test_set_y, y_pred_test.ravel())

pcc_val = pearsonr(strat_val_set_y, y_pred_val.ravel())[0]
rmse_val = rmse(strat_val_set_y, y_pred_val.ravel())

print('-------------------')
print('Inception Model')
print(f'Test Loss: {loss_test[1]:.3f} Val Loss: {loss_val[1]:.3f}')
print(f'Test PCC: {pcc_test:.3f} Val PCC: {pcc_val:.3f}')
print(f'Test RMSE: {rmse_test:.3f} Val RMSE: {rmse_val:.3f}')

### 5 - Data Analysis

#### 5 - 1 History Curve

In [ ]:
mpl.rcParams.update(mpl.rcParamsDefault)
sns.set_style("darkgrid")

In [ ]:
# History of training for each epoch for Sequential and Residual models.

history_plot(sequential_history, residual_history,
             'Sequential Model', 'Residual Model')

#### 5 - 2 Correlation Plot

In [ ]:
mpl.rcParams.update(mpl.rcParamsDefault)
sns.set_style("darkgrid")

In [ ]:
sequential_model = tf.keras.models.load_model('sequential_model.h5')

In [ ]:
# Plotting scatter and histogram plots jointly for Sequential model.

correlation_plot(test_set_y, sequential_model.predict(test_set_x).ravel())

#### 5 - 3 Pearson's Correlation and RMSE Bar Chart

In [ ]:
# Sperate Core set to their clusters id.

target_clusters = {}

for i in range(1, 58):
    target_clusters[i] = []

with open('CoreSet.dat') as file:

    for item in file.readlines():

        item = item.split()

        if item[0] != '#':

            target_clusters[int(item[-1])].append(item[0])

In [ ]:
def pcc_rmse_core_set_cluster(model, pdbid_list):

    x_cluster = np.array([refined_set_data[key]
                         for key in pdbid_list]).reshape(5, 60, 100, 1)
    y_cluster = core_set.set_index('pdbid').loc[pdbid_list].to_numpy()

    y_cluster_pred = model.predict(x_cluster)

    pcc_cluster = pearsonr(y_cluster.ravel(), y_cluster_pred.ravel())[0]
    rmse_cluster = rmse(y_cluster.ravel(), y_cluster_pred.ravel())

    return pcc_cluster, rmse_cluster

In [ ]:
sequential_model = tf.keras.models.load_model('sequential_model.h5')

In [ ]:
pcc_clusters = []
rmse_clusters = []

for item in range(1, 58):

    pcc_cluster, rmse_cluster = pcc_rmse_core_set_cluster(
        sequential_model, target_clusters[item])
    pcc_clusters.append((item, pcc_cluster))
    rmse_clusters.append((item, rmse_cluster))

In [ ]:
pcc_clusters = sorted(pcc_clusters, key=lambda x: x[1], reverse=True)
rmse_clusters = sorted(rmse_clusters, key=lambda x: x[1], reverse=False)

In [ ]:
# Plot Pearson's correlation bar chart for each Core set cluster id.

bar_chart(pcc_clusters, 'summer', '$R_{P}$', 'pcc_bar.png')

In [ ]:
# Plot RMSE bar chart for each Core set cluster id.

bar_chart(rmse_clusters, 'winter', 'RMSE', 'rmse_bar.png')